### MySQL에서 video id를 가져온 후 video Comment get

In [32]:
import numpy as np
import MySQLdb
import pandas as pd
import re
import requests
import schedule
import time

from datetime import datetime
from sqlalchemy import create_engine
from tqdm import tqdm

#### MySql

In [2]:
# connect mysql & Database 
conn = MySQLdb.connect(host="localhost", 
                       port=3306, 
                       user='root', 
                       password='******',
                       database='singAgain_db',
                       charset='utf8')

cursor = conn.cursor()

In [10]:
# create table
#sql = """CREATE TABLE comment_info (currentTime DATETIME, videoId VARCHAR(255), commentTime DATETIME, 
#        comment TEXT, commentLike INT)"""
#cursor.execute(sql)
#conn.commit()

### Get video Info by video ID

In [5]:
# Get video ID from DB
sql = "Select * from video_id_list"
cursor.execute(sql)
video_id_list = cursor.fetchall()

### Get video comment by video ID

In [6]:
# api id
my_id = '******************************'

In [50]:
def get_comment_info(id_list):
    comment_info_list = []
    for one_id in tqdm(id_list):
        # 비디오 정보
        comment_url = f"https://www.googleapis.com/youtube/v3/commentThreads?part=id,snippet&videoId={one_id[0]}&key={my_id}&maxResults=100&order=relevance"
        result = requests.get(comment_url)
        comment_json = result.json()
        
        try:
            for item in comment_json['items']:
                c_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                video_id = one_id[0]
                comment_org = item['snippet']['topLevelComment']['snippet']['textOriginal']
                comment = re.sub("[^ㄱ-힣1-9 ]+", " ", comment_org)
                comment_like = item['snippet']['topLevelComment']['snippet']['likeCount']

                comment_time = item['snippet']['topLevelComment']['snippet']['publishedAt']  
                comment_ptime = comment_time.replace("T", " ").replace("Z", "")

                comment_info_list.append([c_time, video_id, comment_ptime, comment, comment_like])
        except:
            continue
        i = 0
        while i < 9:
            try:
                comment_url = f"https://www.googleapis.com/youtube/v3/commentThreads?part=id,snippet&videoId={one_id[0]}&key={my_id}&maxResults=100&order=relevance&pageToken={comment_json['nextPageToken']}"
            except:
                i += 1
                continue
            result = requests.get(comment_url)
            comment_json = result.json()
            try:
                for item in comment_json['items']:
                    c_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    video_id = one_id[0]

                    comment_org = item['snippet']['topLevelComment']['snippet']['textOriginal']
                    comment = re.sub("[^ㄱ-힣1-9 ]+", " ", comment_org)
                    comment_like = item['snippet']['topLevelComment']['snippet']['likeCount']
                    
                    comment_time = item['snippet']['topLevelComment']['snippet']['publishedAt']  
                    comment_ptime = comment_time.replace("T", " ").replace("Z", "")
                    
                    comment_info_list.append([c_time, video_id, comment_ptime, comment, comment_like])
            except:
                i += 1
                continue
            i += 1
        
    return comment_info_list

In [51]:
# get video id at mysql
def save_comment_info(video_id_list):
    comment_info_list = get_comment_info(video_id_list)
    sql = f"""INSERT INTO comment_info VALUES
                    (%s, %s, %s, %s, %s)"""
    cursor.executemany(sql, comment_info_list)
    conn.commit()

In [52]:
save_comment_info(video_id_list)

100%|██████████| 283/283 [09:40<00:00,  2.05s/it]
